Loading the data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/Fed_Scrape-2015-2023.csv')
df

,Unnamed: 0,Date,Type,Text
0,0,20230412,0,The Federal Reserve on Wednesday released the ...
1,1,20230412,0,The minutes for each regularly scheduled meeti...
2,2,20230412,0,The minutes can be viewed on the Board's website.
3,3,20230412,0,"For media inquiries, e-mail [email protected] ..."
4,4,20230412,0,Minutes of the Federal Open Market Committee\r...
...,...,...,...,...
9969,9969,20150107,0,\r\n The Federal Reserve Board and the F...
9970,9970,20150107,0,\r\n The minutes for each regularly sche...
9971,9971,20150107,0,\r\n FOMC minutes can be viewed on the B...
9972,9972,20150107,0,\nMinutes of the Federal Open Market Committee...


Using TF-IDF (Extractive summarization)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Download NLTK data
nltk.download('punkt')

# Ensure punkt tokenizer is downloaded
nltk.download('punkt_tab', raise_on_error=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:

inp_text = df['Text'][100]

out_text = ""

def summarize_meeting(text, num_sentences=3):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Convert sentences into numerical representations using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(sentences)

    # Calculate sentence scores based on TF-IDF values
    sentence_scores = np.sum(X.toarray(), axis=1)

    # Get the top N sentences with the highest scores
    top_sentence_indices = sentence_scores.argsort()[-num_sentences:][::-1]

    # Sort the selected sentences back in their original order
    top_sentence_indices = sorted(top_sentence_indices)
    summary = ' '.join([sentences[i] for i in top_sentence_indices])

    return summary

if __name__ == "__main__":
    text = inp_text

    summary = summarize_meeting(text)
    out_text = summary
    print("Summary:")
    print(summary)


Summary:
In particular, they noted that revisions to the price data had indicated less disinflation at the end of last year than had been previously reported and that inflation was still quite elevated. Participants noted that, on a 12-month basis, core goods price inflation declined as supply chains continued to improve, but the pace of the decline had slowed, highlighting the still uncertain nature of the disinflationary process. Participants expected that housing services inflation would likely begin to slow in coming months, reflecting continued smaller increases, or potentially declines, in rents on new leases.


Model evaluation using ROUGE score and BERT score

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5eb851d255ccb4bdfbc10d2e0adcbe39d1065c4d024ef77f11cb0e2b886ec964
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def evaluate(generated_summary, reference_summary):
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Compute ROUGE scores
    scores = scorer.score(reference_summary, generated_summary)

    # Print Results
    print("\n🔹 ROUGE Scores:")
    print(f"ROUGE-1  (Unigrams)  -> Precision: {scores['rouge1'].precision:.4f}, Recall: {scores['rouge1'].recall:.4f}, F1-score: {scores['rouge1'].fmeasure:.4f}")
    print(f"ROUGE-2  (Bigrams)   -> Precision: {scores['rouge2'].precision:.4f}, Recall: {scores['rouge2'].recall:.4f}, F1-score: {scores['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L  (LCS)       -> Precision: {scores['rougeL'].precision:.4f}, Recall: {scores['rougeL'].recall:.4f}, F1-score: {scores['rougeL'].fmeasure:.4f}")




In [ ]:
!pip install bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from bert_score import score

def bertscore(original_text, summary_text, model_type='bert-base-uncased'):
    P, R, F1 = score([summary_text], [original_text], lang='en', model_type=model_type)

    print(f"\n📊 BERTScore Evaluation:")
    print(f"🔹 Precision: {P.item():.4f}")
    print(f"🔹 Recall:    {R.item():.4f}")
    print(f"🔹 F1 Score:  {F1.item():.4f}")

    return {
        'precision': P.item(),
        'recall': R.item(),
        'f1': F1.item()
    }


Ratio of input and output text lengths and evaluation

In [ ]:
len(inp_text)/len(out_text)

2.804560260586319

In [ ]:
evaluate(out_text,inp_text)


🔹 ROUGE Scores:
ROUGE-1  (Unigrams)  -> Precision: 1.0000, Recall: 0.3760, F1-score: 0.5465
ROUGE-2  (Bigrams)   -> Precision: 1.0000, Recall: 0.3735, F1-score: 0.5439
ROUGE-L  (LCS)       -> Precision: 1.0000, Recall: 0.3760, F1-score: 0.5465


Using BERT (Extractive summarization)

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
inp_text1 = df['Text'][100]
out_text1 = ""
def encode_sentence(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def summarize_bert(text, num_sentences=3):
    # Load Pre-trained BERT model
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    # Split text into sentences
    sentences = sent_tokenize(text)

    # Get BERT embeddings for each sentence
    sentence_embeddings = np.array([encode_sentence(s, model, tokenizer) for s in sentences])

    # Get document-level embedding (mean of all sentences)
    doc_embedding = np.mean(sentence_embeddings, axis=0).reshape(1, -1)

    # Calculate sentence importance using cosine similarity
    sentence_scores = cosine_similarity(sentence_embeddings, doc_embedding).flatten()

    # Select top N sentences
    top_n_idx = sentence_scores.argsort()[-num_sentences:][::-1]
    top_n_idx = sorted(top_n_idx)  # Preserve sentence order

    # Generate Summary
    summary = ' '.join([sentences[i] for i in top_n_idx])
    return summary


if __name__ == "__main__":
    text = inp_text1


    summary = summarize_bert(text)
    out_text1 =summary
    print("Summary:")
    print(summary)


Summary:
Participants commented that recent inflation data indicated slower-than-expected progress on disinflation. Participants noted that, on a 12-month basis, core goods price inflation declined as supply chains continued to improve, but the pace of the decline had slowed, highlighting the still uncertain nature of the disinflationary process. Additionally, participants observed that indicators of short-term inflation expectations from surveys of households and businesses had come down further, while longer-term inflation expectations remained well anchored.


In [ ]:
inp_text1

"With inflation still well above the Committee's longer-run goal of 2 percent, participants agreed that inflation was unacceptably high. Participants commented that recent inflation data indicated slower-than-expected progress on disinflation. In particular, they noted that revisions to the price data had indicated less disinflation at the end of last year than had been previously reported and that inflation was still quite elevated. Participants noted that, on a 12-month basis, core goods price inflation declined as supply chains continued to improve, but the pace of the decline had slowed, highlighting the still uncertain nature of the disinflationary process. Participants expected that housing services inflation would likely begin to slow in coming months, reflecting continued smaller increases, or potentially declines, in rents on new leases. Regarding prices for core services excluding housing, participants agreed that there was little evidence pointing to disinflation in this com

In [ ]:
out_text1

'Participants commented that recent inflation data indicated slower-than-expected progress on disinflation. Participants noted that, on a 12-month basis, core goods price inflation declined as supply chains continued to improve, but the pace of the decline had slowed, highlighting the still uncertain nature of the disinflationary process. Additionally, participants observed that indicators of short-term inflation expectations from surveys of households and businesses had come down further, while longer-term inflation expectations remained well anchored.'

Ration of input and output texts and evaluation

In [ ]:
len(inp_text1)/len(out_text1)

3.086021505376344

In [ ]:
evaluate(out_text1,inp_text1)


🔹 ROUGE Scores:
ROUGE-1  (Unigrams)  -> Precision: 1.0000, Recall: 0.3080, F1-score: 0.4709
ROUGE-2  (Bigrams)   -> Precision: 0.9737, Recall: 0.2972, F1-score: 0.4554
ROUGE-L  (LCS)       -> Precision: 1.0000, Recall: 0.3080, F1-score: 0.4709


BART(Abstractive summarization)

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

def summarize_with_bart(text, max_summary_length=300, min_summary_length=100):
    model_name = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Tokenize and chunk if input exceeds BART’s limit (1024 tokens)
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(
        inputs,
        max_length=max_summary_length,     # You control the summary length here
        min_length=min_summary_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
def split_into_chunks(text, max_tokens=1024):
    sentences = text.split('. ')
    chunks, chunk = [], ''
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

    for sentence in sentences:
        tentative_chunk = chunk + sentence + '. '
        if len(tokenizer.encode(tentative_chunk)) < max_tokens:
            chunk = tentative_chunk
        else:
            chunks.append(chunk.strip())
            chunk = sentence + '. '
    if chunk:
        chunks.append(chunk.strip())
    return chunks


In [ ]:
inp_text1 = df['Text'][100]
out_text1 = ""

In [ ]:
inp_text1

"With inflation still well above the Committee's longer-run goal of 2 percent, participants agreed that inflation was unacceptably high. Participants commented that recent inflation data indicated slower-than-expected progress on disinflation. In particular, they noted that revisions to the price data had indicated less disinflation at the end of last year than had been previously reported and that inflation was still quite elevated. Participants noted that, on a 12-month basis, core goods price inflation declined as supply chains continued to improve, but the pace of the decline had slowed, highlighting the still uncertain nature of the disinflationary process. Participants expected that housing services inflation would likely begin to slow in coming months, reflecting continued smaller increases, or potentially declines, in rents on new leases. Regarding prices for core services excluding housing, participants agreed that there was little evidence pointing to disinflation in this com

In [ ]:
mom_text = inp_text1

chunks = split_into_chunks(mom_text)
summaries = [summarize_with_bart(chunk, max_summary_length=300, min_summary_length=100) for chunk in chunks]
final_summary = ' '.join(summaries)
out_text = final_summary
print("📋 Final Summary:\n", final_summary)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

📋 Final Summary:
 With inflation still well above the Committee's longer-run goal of 2 percent, participants agreed that inflation was unacceptably high. Participants expected that housing services inflation would likely begin to slow in coming months, reflecting continued smaller increases, or potentially declines, in rents on new leases. Participants generally judged that some more easing in labor market tightness and slowing in nominal wage growth would be necessary for sustained disinflation. They noted that a tightening of credit conditions was likely to weigh on aggregate demand, which could help reduce inflationary pressures.


In [ ]:
out_text

"With inflation still well above the Committee's longer-run goal of 2 percent, participants agreed that inflation was unacceptably high. Participants expected that housing services inflation would likely begin to slow in coming months, reflecting continued smaller increases, or potentially declines, in rents on new leases. Participants generally judged that some more easing in labor market tightness and slowing in nominal wage growth would be necessary for sustained disinflation. They noted that a tightening of credit conditions was likely to weigh on aggregate demand, which could help reduce inflationary pressures."

Ration of input and output text and evaluation

In [ ]:
len(inp_text1)/len(out_text)

2.7684887459807075

In [ ]:
evaluate(out_text,inp_text1)


🔹 ROUGE Scores:
ROUGE-1  (Unigrams)  -> Precision: 1.0000, Recall: 0.3640, F1-score: 0.5337
ROUGE-2  (Bigrams)   -> Precision: 0.9667, Recall: 0.3494, F1-score: 0.5133
ROUGE-L  (LCS)       -> Precision: 1.0000, Recall: 0.3640, F1-score: 0.5337


In [ ]:
bertscore(inp_text1,out_text)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


📊 BERTScore Evaluation:
🔹 Precision: 0.9076
🔹 Recall:    0.7070
🔹 F1 Score:  0.7948


{'precision': 0.9076100587844849,
 'recall': 0.7070016860961914,
 'f1': 0.7948435544967651}

Creating a csv file which compares all 3 models using ROUGE and BERT score

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from tqdm import tqdm
import torch

nltk.download('punkt')

# Load your input CSV
df = pd.read_csv("/content/Fed_Scrape-2015-2023.csv")
texts = df['Text'].dropna().tolist()[100:200]  # use first 100 non-empty entries

# --- TF-IDF Summarizer ---
def tfidf_summarize(text, top_n=3):
    sentences = nltk.sent_tokenize(text)
    if len(sentences) < top_n:
        return text
    tfidf = TfidfVectorizer().fit_transform(sentences)
    scores = tfidf.sum(axis=1).A1
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    summary = ' '.join([sentences[i] for i in top_indices])
    return summary

# --- BERT Summarizer ---
bert_summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=0 if torch.cuda.is_available() else -1)

def bert_summarize(text):
    try:
        return bert_summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    except:
        return text[:150]  # fallback

# --- BART Summarizer ---
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

def bart_summarize(text):
    inputs = bart_tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=200, min_length=60, num_beams=4, length_penalty=2.0, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# --- Evaluation ---
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def evaluate_metrics(reference, generated):
    r = rouge.score(reference, generated)
    _, _, bert_f1 = bert_score([generated], [reference], lang='en')
    return {
        "ROUGE-1": r['rouge1'].fmeasure,
        "ROUGE-L": r['rougeL'].fmeasure,
        "BERTScore": bert_f1.item()
    }

# --- Main Loop ---
results = []

for i, text in enumerate(tqdm(texts, desc="Summarizing")):
    reference = text.strip()

    summaries = {
        "TF-IDF": tfidf_summarize(reference),
        "BERT": bert_summarize(reference),
        "BART": bart_summarize(reference)
    }

    for model_name, summary in summaries.items():
        scores = evaluate_metrics(reference, summary)
        results.append({
            "Index": i + 1,
            "Model": model_name,
            "ROUGE-1": scores["ROUGE-1"],
            "ROUGE-L": scores["ROUGE-L"],
            "BERTScore": scores["BERTScore"],
            "Summary": summary
        })

# --- Save Output ---
out_df = pd.DataFrame(results)
out_df.to_csv("summarization_comparison.csv", index=False)

print("✅ Results saved to summarization_comparison.csv")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cuda:0
Summarizing:   0%|          | 0/100 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Summarizing:   1%|          | 1/100 [00:58<1:35:56, 58.14s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly ini

✅ Results saved to summarization_comparison.csv


In [ ]:
for r in results:
  r['Transcript'] = df['Text'][100+r['Index']-1]

out_df = pd.DataFrame(results)
out_df.to_csv("summarization_comparison2.csv", index=False)

print("✅ Results saved to summarization_comparison.csv")



✅ Results saved to summarization_comparison.csv
